In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from datetime import datetime
now = datetime.now()
formatted_time = now.strftime("%m%d%H%M")
EXP_NAME = f'v8_{formatted_time}'
print(EXP_NAME)

v8_12221346


In [ ]:
import os
import numpy as np
import pandas as pd
folder = '/content/drive/MyDrive/YouniB/data'
all_df =[]
for file_path in os.listdir(folder):
    if file_path.endswith('.json'):
        tmp_df = pd.read_json(os.path.join(folder, file_path))
        tmp_df = tmp_df[['Question','Answer','type','Options','Context','Page','과목명','강의명']]
        tmp_df['Options'] = tmp_df['Options'].replace({'nan': np.nan, 'NaN': np.nan, 'NAN': np.nan}, regex=True)
        all_df.append(tmp_df)

df_adh_m = pd.read_json('/content/drive/MyDrive/YouniB/gptgenerated_M.json')
df_adh_s = pd.read_json('/content/drive/MyDrive/YouniB/gptgenerated_S.json')
df_adh_m['type'] = '객관식'
df_adh_s['type'] = '단답식'

all_df.append(df_adh_s)
all_df.append(df_adh_m)

df = pd.concat(all_df)

df_m = df[df['type']=='객관식']
df_s = df[df['type']=='단답식']

print(f'전체 df의 객관식, 단답식 분포')
display(df['type'].value_counts())

전체 df의 객관식, 단답식 분포


,count
type,
객관식,4362
단답식,4344


In [ ]:
# 중복 행 확인
df_str = df.applymap(lambda x: str(x) if isinstance(x, list) else x)
duplicates = df_str[df_str.duplicated(keep=False)].sort_values(df_str.columns.tolist())
display(duplicates)

del df_str, duplicates

<ipython-input-4-78ad69799d0e>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_str = df.applymap(lambda x: str(x) if isinstance(x, list) else x)


,Question,Answer,type,Options,Context,Page,과목명,강의명
562,What is the term for the reactions in which al...,sequential reactions,단답식,NaN,84 THE MICHAELISMENTEN MODEL The Significance ...,"[31, 35]",서강대학교-생화학1,Chapter8
563,What is the term for the reactions in which al...,sequential reactions,단답식,NaN,84 THE MICHAELISMENTEN MODEL The Significance ...,"[31, 35]",서강대학교-생화학1,Chapter8


In [ ]:
! pip install transformers -qq
! pip install accelerate  -qq
! pip install tiktoken -qq
! pip install einops  -qq
! pip install scipy  -qq
! pip install transformers_stream_generator -qq
! pip install peft -qq
! pip install deepspeed  -qq
! pip install bitsandbytes -qq
! pip install trl -qq
! pip install auto-gptq optimum -qq

In [ ]:
! pip install -U bitsandbytes -qq


In [ ]:
from huggingface_hub import login
secret_value_0 = '<Huggingface API key>'
user = login(token=secret_value_0)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
import warnings
warnings.filterwarnings("ignore")

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

orig_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.1-8B-Instruct",
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")
tokenizer.pad_token = tokenizer.eos_token
orig_model.config.pad_token_id = tokenizer.pad_token_id

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
orig_model

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline


In [ ]:
from peft import LoraConfig, get_peft_model, PeftModel, PeftConfig

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

peft_model = get_peft_model(orig_model, lora_config)
peft_model.print_trainable_parameters()
peft_model

trainable params: 6,815,744 || all params: 8,037,076,992 || trainable%: 0.0848


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj):

In [ ]:
df_m

,Question,Answer,type,Options,Context,Page,과목명,강의명
0,Which of the following is NOT a characteristic...,C,객관식,"[A. They are composed of G-actin subunits., B....",Actin filaments 1 Actin exists as a globular m...,"[100, 104]",부산대학교-세포생물학특론,10
1,What is the term for the globular monomer form...,C,객관식,"[A. F-actin, B. M-actin, C. G-actin, D. A-actin]",Actin filaments 1 Actin exists as a globular m...,"[100, 104]",부산대학교-세포생물학특론,10
4,Which of the following is NOT a characteristic...,C,객관식,"[A. They are composed of G-actin subunits., B....",The three phases of in vitro Gactin polymeriza...,"[103, 107]",부산대학교-세포생물학특론,10
5,What is the term for the globular monomer form...,C,객관식,"[A. F-actin, B. M-actin, C. G-actin, D. A-actin]",The three phases of in vitro Gactin polymeriza...,"[103, 107]",부산대학교-세포생물학특론,10
8,Which end of a myosin-decorated actin filament...,B,객관식,"[A. (-) end, B. (+) end, C. Both ends grow equ...",Figure 1710 Actin treadmilling Actin treadmill...,"[106, 110]",부산대학교-세포생물학특론,10
...,...,...,...,...,...,...,...,...
2101,Which compound is essential for yeast fermenta...,B,객관식,"[A. Oxygen, B. Glucose, C. Ammonium, D. Glycer...",계와네트워크분석 Spread of viruses The spread of virus...,"[16, 20]",건국대학교-시스템생물학기초,11
2102,What is a characteristic feature of conditiona...,A,객관식,[A. Mutations expressed only under specific co...,Human protein network of PKB What do we get an...,"[19, 23]",건국대학교-시스템생물학기초,11
2103,Which phase of the yeast cell cycle is most se...,A,객관식,"[A. G1 phase, B. S phase, C. M phase, D. G2 ph...",Human protein network of PKB What do we get an...,"[19, 23]",건국대학교-시스템생물학기초,11
2104,What is the role of the spindle assembly check...,B,객관식,"[A. Initiates DNA replication, B. Ensures prop...",Biological Networks and Complex Systems UNCOVE...,"[22, 25]",건국대학교-시스템생물학기초,11


In [ ]:
! pip install datasets

In [ ]:
A = tokenizer(
        '33',
        truncation=True,
        max_length=4,
        padding="max_length",
        add_special_tokens=False
        )
A

{'input_ids': [1644, 128009, 128009, 128009], 'attention_mask': [1, 0, 0, 0]}

In [ ]:
from transformers import Trainer, TrainingArguments
from datasets import Dataset
import torch

M_context_prompt_format = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an expert in generating quizzes.
Given the context regarding biology, create a single multiple-choice question that accurately tests knowledge based on the context.

Context: {0}

If you decide to provide a quiz, you MUST strictly follow the format below:
The format for the multiple-choice quiz is:

  Question: <Your Question Here>
  Options:
  <Option A>
  <Option B>
  <Option C>
  <Option D>
  Answer: <Correct Option>

Additional rules:
1. Include exactly four options in the \"Options\" list.
2. Avoid repeating any information across the options.
3. Ensure that one and only one option is correct.
4. Ensure that the question and options are directly related to the provided context.
5. You SHOULD NOT include any other text in the response.
<|eot_id|><|start_header_id|>user<|end_header_id|>

Now create a single multiple-choice question.
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""

S_context_prompt_format = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an expert in generating quizzes.
Given the context regarding biology, create a single short-answer question that accurately tests knowledge based on the context.

Context: {0}

If you decide to provide a quiz, you MUST strictly follow the format below:

Question: <Your Question Here>
Answer: <Correct Answer>

Additional rules:
1. Ensure the question and answer are directly related to the provided context.
2. Include only one correct answer.
3. You SHOULD NOT include any other text in the response.
<|eot_id|><|start_header_id|>user<|end_header_id|>

Now create a single short-answer question.
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""


# 1. 데이터 로드
def prepare_data(df):
    data = []
    for idx, row in df.iterrows():
        if row['type'] == '객관식':
            prompt_template = M_context_prompt_format
        elif row['type'] == '단답식':
            prompt_template = S_context_prompt_format
        else:
            print(f'{idx}번째 행의 Type 오류')

        context = row['Context']
        prompt = prompt_template.format(context)

        if row['type'] == '객관식':
            output_text = (
                f"Question: {row['Question']}\n"
                "Options:\n"
                f"  {row['Options'][0]}\n"
                f"  {row['Options'][1]}\n"
                f"  {row['Options'][2]}\n"
                f"  {row['Options'][3]}\n"
                f"Answer: {row['Answer']}"
            )
        else:
            output_text = f"Question: {row['Question']}\nAnswer: {row['Answer']}"

        data.append({"input_text": prompt, "context": context, "output_text": output_text})

    return data

data = prepare_data(df)
dataset = Dataset.from_pandas(pd.DataFrame(data))

#############################################################################

# 2. Prompt Prefix 길이 미리 계산
def calculate_prompt_length(prompt_template):
    placeholder_context = " "
    full_prompt = prompt_template.format(placeholder_context)
    tokenized_prompt = tokenizer(
        full_prompt,
        truncation=False,
        add_special_tokens=False
    )["input_ids"]
    return len(tokenized_prompt)

M_prompt_length = calculate_prompt_length(M_context_prompt_format)
S_prompt_length = calculate_prompt_length(S_context_prompt_format)
print(f'M_prompt_length: {M_prompt_length}')
print(f'S_prompt_length: {S_prompt_length}')

#############################################################################

# 3. 전처리 함수
def preprocess_data(examples):
    MAX_TOKENS = 3200

    model_inputs = {
        "input_ids": [],
        "attention_mask": [],
        "labels": []
    }

    for input_text, output_text in zip(examples["input_text"], examples["output_text"]):

        # Context가 삽입된 위치를 파악
        context_start_index = input_text.find("Context: ") + len("Context: ")
        if context_start_index == -1:
            raise ValueError("Prompt format is invalid. 'Context:' not found.")

        # 프롬프트 앞부분과 뒷부분 분리
        prompt_prefix = input_text[:context_start_index]  # "Context:" 포함
        context_start_index = context_start_index  # Context의 실제 텍스트 시작 위치
        context_end_index = input_text.find("Now create", context_start_index)
        if context_end_index == -1:
            raise ValueError("Prompt format is invalid. 'Now create' not found after context.")

        # Context 추출
        context = input_text[context_start_index:context_end_index].strip()
        prompt_suffix = input_text[context_end_index:].strip()

        # 프롬프트 앞/뒤의 고정 길이를 계산
        prompt_prefix_ids = tokenizer(
            prompt_prefix,
            truncation=False,
            add_special_tokens=False
        )["input_ids"]
        prompt_suffix_ids = tokenizer(
            prompt_suffix,
            truncation=False,
            add_special_tokens=False
        )["input_ids"]

        fixed_prompt_length = len(prompt_prefix_ids) + len(prompt_suffix_ids)

        # Context에 할당된 최대 길이 계산
        max_context_length = MAX_TOKENS - fixed_prompt_length

        # Context 토큰화 및 잘라내기
        context_ids = tokenizer(
            context,
            truncation=True,
            max_length=max_context_length,
            padding=False,
            add_special_tokens=False
        )["input_ids"]

        # 최종 프롬프트 병합
        full_prompt_ids = prompt_prefix_ids + context_ids + prompt_suffix_ids
        attention_mask = [1] * len(full_prompt_ids)

        # 출력 텍스트 토큰화
        output_ids = tokenizer(
            output_text,
            truncation=True,
            max_length=192,
            padding="max_length",
            add_special_tokens=False
        )["input_ids"]

        # 입력과 출력 병합
        combined_ids = full_prompt_ids + output_ids
        combined_mask = attention_mask + [1] * len(output_ids)

        # 라벨 생성
        labels = [-100] * len(full_prompt_ids) + [
            token if token != tokenizer.pad_token_id else -100 for token in output_ids
        ]

        # 결과 저장
        model_inputs["input_ids"].append(combined_ids)
        model_inputs["attention_mask"].append(combined_mask)
        model_inputs["labels"].append(labels)

    return model_inputs

tokenized_dataset = dataset.map(preprocess_data, batched=True)

M_prompt_length: 185
S_prompt_length: 123


Map:   0%|          | 0/8706 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir=f"/content/drive/MyDrive/YouniB/FT/finetuned_model_{EXP_NAME}",
    save_steps=1000,
    logging_dir=f"/content/drive/MyDrive/YouniB/FT/finetuned_model/logs/{EXP_NAME}",
    logging_steps=10,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=10,
    learning_rate=2e-5,
    num_train_epochs=2,
    fp16=True,
    seed=42,
    report_to="none",
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    tokenizer=tokenizer,
)

trainer.train()

trainer.save_model(f"/content/drive/MyDrive/YouniB/FT/finetuned_model_{EXP_NAME}")

[2024-12-22 13:48:23,467] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Step,Training Loss
10,2.734500
20,2.657500
30,2.548100
40,2.467500
50,2.235800
60,2.256400
70,2.106700
80,1.898200
90,1.839200
100,1.840000


In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
log_df = pd.DataFrame(trainer.state.log_history)


In [ ]:
loss_df = log_df[['loss','train_loss','step']]
loss_df

,loss,train_loss,step
0,2.7345,NaN,10
1,2.6575,NaN,20
2,2.5481,NaN,30
3,2.4675,NaN,40
4,2.2358,NaN,50
5,2.2564,NaN,60
6,2.1067,NaN,70
7,1.8982,NaN,80
8,1.8392,NaN,90
9,1.8400,NaN,100


In [ ]:
log_df.iloc[-1]

,174
loss,NaN
grad_norm,NaN
learning_rate,NaN
epoch,1.998162e+00
step,1.740000e+03
train_runtime,5.738933e+03
train_samples_per_second,3.034000e+00
train_steps_per_second,3.030000e-01
total_flos,5.636981e+17
train_loss,1.709854e+00


In [ ]:
import gc,torch
gc.collect()
torch.cuda.empty_cache()

# 저장된 불러와서 추론해보기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers -qq
! pip install accelerate  -qq
! pip install tiktoken -qq
! pip install einops  -qq
! pip install scipy  -qq
! pip install transformers_stream_generator -qq
! pip install  peft -qq
! pip install deepspeed  -qq
! pip install bitsandbytes -qq
! pip install trl -qq
!pip install auto-gptq optimum -qq

In [ ]:
from datetime import datetime
print(datetime.now())
import transformers
transformers.__version__

2024-12-22 15:26:33.286974


'4.47.1'

In [ ]:
! pip install -U bitsandbytes


In [ ]:
from huggingface_hub import login
secret_value_0 = '<HUGGINGFACE API KEY>'
user = login(token=secret_value_0)

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
import warnings
warnings.filterwarnings("ignore")

# 4-bit 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

orig_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.1-8B-Instruct",
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct")
tokenizer.pad_token = tokenizer.eos_token
orig_model.config.pad_token_id = tokenizer.pad_token_id

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
from peft import LoraConfig, get_peft_model, PeftModel, PeftConfig

# 모델과 토크나이저 경로 설정
model_path = f"/content/drive/MyDrive/[개인] YouniB/sehyeon/FT/finetuned_model_{EXP_NAME}"

# 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained(model_path)

# PEFT 모델 불러오기
inf_model = PeftModel.from_pretrained(orig_model, model_path)
final_model = inf_model.merge_and_unload()
final_model = final_model.eval()


In [ ]:
context_ex = 'Actin filaments 1 Actin exists as a globular monomer called Gactin and as a filamentous polymer called Factin which is a linear chain of Gactin subunits Each actin molecule contains a ion complexed with either ATP or ADP The importance of the interconversion between the ATP and the ADP forms of actin is discussed later From the results of xray diffraction studies of actin filaments and the actin monomer structure shown in Figure 175a scientists have determined that the subunits in an actin filament are arranged in a helical structure Figure 175b In this arrangement the filament can be considered as two helical strands wound around each other Each subunit in the structure contacts one subunit above one below in one strand and two in the other strand When Factin is negatively stained with uranyl acetate for electron microscopy it appears as a twisted string whose diameter varies between 7 and 9 nm Figure 17 5c 2 Factin has structural and functional polarity All subuits in an actin filament are oriented the same way Consequently a filament exhibits polarity that is one end differs from the other As we will see one end of the filament is favored for the addition of actin subunits and is designated the end whereas the other end is favored for subunit dissociation designated the end At the end the ATPbinding cleft of the terminal actin subunit contacts the neighboring subunit whereas on the end the cleft is exposed to the surrounding solution Figure 175b 3 The polarity of actin filaments can be demonstrated by electron microscopy in decoration experiments which exploit the ability of the motor protein myosin to bind specifically to actin filaments In this type of experiment an excess of myosin S1 the actinbinding head domain of myosin is mixed with actin filaments and binding is permitted to take place Myosin attaches to the sides of a filament with a slight tilt When all the actin subunits are bound by myosin the filament appears decorated with arrowheads that all point toward one end of the filament Figure 176 Figure 177 The three phases of in vitro Gactin polymerization The three phases of in vitro Gactin polymerization 1 The mechanism of actin assembly has been studied extensively Remarkably one can purify Gactin at a high protein concentration without it forming filamentsprovided it is maintained in a buffer with ATP and low levels of cations However as we saw earlier if the cation level is increased eg to 100mM and 2mM Gactin will polymerize with the kinetics of the reaction depending on the starting concentration of Gactin The polymerization of pure Gactin in vitro proceeds in three sequential phases Figure 17 7a The nucleation phase is marked by a lag period in which Gactin subunits combine into two or three subunits When the oligomer reaches three subunits in length it can act as a seed or nucleus for the next phase During the elongation phase the short oligomer rapidly increase in length by the addition of actin monomers to both of its ends As Factin filaments grow the concentration of Gactin monomers decreases until equilibrium is reached between the filament ends and monomers and a steady state is reached In the steadystate phase Gactin monomers exchange with subunits at the filament ends but there is no net change in the total length of filaments 2 The kinetic curves in Figure 177b c show the state of filament mass during each phase of polymerization In Figure 177c we see that the lag period is due to nucleation because it can be eliminated by the addition of a small number of Factin nuclei to the solution of Gactin Figure 178 Determination of filament formation by actin concentration'
prompt_ex = M_context_prompt_format.format(context_ex)

a = preprocess_data({'input_text':[prompt_ex], 'output_text':['']})
b = {x: torch.tensor(y).to('cuda') for x, y in a.items() if x != 'labels'}
for i, x in b.items():
    print(i,x)
    if i == 'input_ids':
        print(tokenizer.decode(x[0]))

gen_txt = final_model.generate(
    **b,
    max_new_tokens=200,
    no_repeat_ngram_size=3,
    repetition_penalty=1.2,
    temperature=0.95,
    do_sample=True,
    top_p=0.95,
    num_return_sequences=1
)
print(tokenizer.decode(gen_txt[0]))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


input_ids tensor([[128000, 128006,   9125,  ..., 128009, 128009, 128009]],
       device='cuda:0')
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an expert in generating quizzes.
Given the context regarding biology, create a single multiple-choice question that accurately tests knowledge based on the context.

Context: Actin filaments 1 Actin exists as a globular monomer called Gactin and as a filamentous polymer called Factin which is a linear chain of Gactin subunits Each actin molecule contains a ion complexed with either ATP or ADP The importance of the interconversion between the ATP and the ADP forms of actin is discussed later From the results of xray diffraction studies of actin filaments and the actin monomer structure shown in Figure 175a scientists have determined that the subunits in an actin filament are arranged in a helical structure Figure 175b In this arrangement the filament can be considered as two helical strands wound around each other Each sub

In [ ]:
from datetime import datetime
datetime.now()

datetime.datetime(2024, 12, 22, 15, 27, 21, 565767)